In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal
from utils import encode, ofdm, channels, audio, decode

cwd = os.getcwd()

In [2]:
DFT_LENGTH = 8192
CP_LENGTH = 1024
silence_duration = 1
chirp_duration = 1
fs = 48000

low_freq = 20
high_freq = 10000
double_chirp = False

In [3]:
# Generate a bit array
new = True
if new:
    np.random.seed(0)
    encoded = np.random.randint(2, size=10 * fs)
    np.savetxt('encoded', encoded, fmt='%i')

In [4]:
# Load a saved bit array
encoded = np.loadtxt('encoded', dtype=np.int32)
qpsk_symbols = encode.qpsk_encode(encoded)
shifted = ofdm.subcarrier_shift_gaussian(qpsk_symbols, DFT_LENGTH, fs, low_freq, high_freq, 1)


577395


In [5]:
ofdm_data = ofdm.symbols_to_ofdm(shifted, DFT_LENGTH, CP_LENGTH)
print(ofdm_data.size)
print(ofdm_data[:10])
audio.signal_to_wav(np.real(ofdm_data), fs, 'ofdm_tester', 'audio_files')


1299456
[-0.0104164 +0.j -0.01013846+0.j  0.04766509+0.j -0.00720886+0.j
 -0.00423713+0.j  0.01903234+0.j -0.00581751+0.j  0.00668477+0.j
 -0.03407705+0.j -0.00192746+0.j]


In [6]:
# Send through Octagon
octagon = channels.octagon(os.path.join(cwd, "audio_files"), fs)
received = channels.channel_output(octagon, np.real(ofdm_data))
fft = ofdm.ofdm_to_fourier(received[:ofdm_data.size], DFT_LENGTH, CP_LENGTH)
discarded = ofdm.subcarrier_extract(fft, DFT_LENGTH, fs, low_freq, high_freq)
deconvolved = ofdm.deconvolve(discarded, octagon, DFT_LENGTH, fs, low_freq, high_freq)
decoded = decode.qpsk_decode(deconvolved)
print(encoded.size, decoded.size)
print("source:\n", encoded[:30])
print("decoded:\n", decoded[:30])



480000 480246
source:
 [0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0]
decoded:
 [1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0]


In [7]:
equality = encoded == decoded[:encoded.size]
print("correct rate:\n", np.sum(equality.astype(int))/equality.size)

correct rate:
 0.74638125
